In [ ]:
import math as m

def CreateBlockMatrix(dimensions, blockshape, blocklist):
    '''Create one matrix out of several specified blocks'''
    return block_matrix(dimensions[0],dimensions[1], [blocklist[t] for t in blockshape])



def is_zeroone(matrix):
    '''Check if a matrix is a zero-one matrix'''
    for row in matrix:
        for entry in row:
            if entry not in {0,1}:
                return False

    return True



def switchingblock(level, blocknumber, sort):
    '''Create a regular orthogonal permutation-based matrix of a certain level, number of blocks and sort'''
    J = ones_matrix(level)
    O = zero_matrix(level)
    Y = level*identity_matrix(level)- J

    def rotate(l):
        return [l[-1]] + l[:-1]

    baserow = [0,2] + [1]*(blocknumber-2)
    shapematrix = []
    for _ in range(blocknumber):
        shapematrix = shapematrix + baserow
        baserow = rotate(baserow)


    if sort == 'wqh':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [Y,O,J])/level
    if sort == 'gm':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [-Y,O,J])/level


def getvertices(n, S):
    '''Gives a list where the numbers in S are moved to the front, which will be used as a permutation'''
    complement = set(range(1,n+1)) - set(S)
    return S + list(complement)



def symmetrymatrices(n, blocks, level, sort, recursionlevel):
    '''This function recursively finds all permutations (as matrices) within the switching set.
    At each recursion level it takes any set of level size and puts it to the front. We do this, because the switching matrix has a symmetry group induced by rotating the blocks and rotating within the blocks. Therefore it only matters what the set of vertices in each block are, so we pick combinations. First we pick the second to last block, then at each recursion level we go back until we pick the first block.For each block we may only pick from [1,....,blocks*level-1]. Thus ensuring that the last block has the highest number in it. This beats the symmetry of the block rotation.'''

    #I could remove n and replace by blocks*level just have to add identity matrix to P1
    if blocks == 2 and sort == 'gm':
        return [identity_matrix(n)]
    if recursionlevel == 1:
        return [identity_matrix(n)]

    matrices= []
    for X in Combinations(range(level*(blocks-recursionlevel)+1, blocks*level), level):
        Xmatrix = Permutation(getvertices(n, X)).to_matrix()
        for P in symmetrymatrices(n, blocks, level, sort, recursionlevel-1):
            # As the matrices will be transposed (on the left) in bruteswitching. The way to ensure that the highest level does the second to last block is to multiply Xmatrix on the left. To be honest I don't think it matters, you will get the same set of symmetrymatrices either way, but at least this way I can explain it.
            matrices.append(Xmatrix*P)

    return matrices


def Genswitchingset(level, blocks, sort):
    '''Create all labelled graphs (up to complementation) that form a switching set for the Q(level,blocks, sort)-switching method
        up to permutation that leaves Q fixed.'''
    switchingset = []

    Q = switchingblock(level, blocks, sort)
    Permlist = symmetrymatrices(blocks*level, blocks, level, sort, blocks)
    n = level*blocks
    for G in graphs.nauty_geng(str(n) + ' 0:' + str(m.floor(n*(n-1)/4))):
        A = G.adjacency_matrix()
        for P in Permlist:
            B = Q.transpose()*P.transpose()*A*P*Q
            if is_zeroone(B):
                switchingset.append(P.transpose()*A*P)
                break

    return [k for k, _ in its.groupby(sorted(switchingset))]


#Compute Aut_Q(Gamma) for each switching graph of GM8

GM8ss = Genswitchingset(2,4, 'gm')
graphlist = set()

for matrix in GM8ss:
    G =Graph(matrix)
    graphlist.add(G.canonical_label().copy(immutable = True))
    graphlist.add(G.complement().canonical_label().copy(immutable=True))

agammalist = [len(G.automorphism_group()) for G in graphlist]

print(agammalist)
print(sum([1/agamma for agamma in agammalist]))

#Compute Aut_Q(Gamma) for each switching graph of WQH8

WQH8ss = Genswitchingset(2,4, 'wqh')
graphlist = set()

for matrix in WQH8ss:
    G =Graph(matrix)
    graphlist.add(G.canonical_label().copy(immutable = True))
    graphlist.add(G.complement().canonical_label().copy(immutable=True))

agammalist = [len(G.automorphism_group()) for G in graphlist]

print(agammalist)
print(sum([1/agamma for agamma in agammalist]))
